<a href="https://colab.research.google.com/github/Mithaamalia13/Kecerdasan-Buatan/blob/main/SVM%20Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import library

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
from sklearn.svm import SVR
import plotly.express as px
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.linear_model import LinearRegression,Ridge,Lasso
import plotly.graph_objects as go

In [ ]:
# Mengambil data

df = pd.read_csv("beras_kaltim.csv")

In [ ]:
# Informasi Data

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 429 entries, 0 to 428
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   Komoditas(Rp)                  429 non-null    object
 1   Beras                          429 non-null    int64 
 2   Beras Kualitas Bawah I (kg)    429 non-null    int64 
 3   Beras Kualitas Bawah II (kg)   429 non-null    int64 
 4   Beras Kualitas Medium I (kg)   429 non-null    int64 
 5   Beras Kualitas Medium II (kg)  429 non-null    int64 
 6   Beras Kualitas Super I (kg)    429 non-null    int64 
 7   Beras Kualitas Super II (kg)   429 non-null    int64 
dtypes: int64(7), object(1)
memory usage: 26.9+ KB


In [ ]:
# Merubah tipe data Komoditas menjadi datetime

df['Komoditas(Rp)'] = pd.to_datetime(df['Komoditas(Rp)'])

In [ ]:
# Melakukan grup pada data beras kualitas 1 berdasarkan komoditas

datewise=df.groupby(["Komoditas(Rp)"]).agg({"Beras Kualitas Bawah I (kg)":'sum'})
datewise["Days Since"]=datewise.index-datewise.index.min()

In [ ]:
df.describe()

,Beras,Beras Kualitas Bawah I (kg),Beras Kualitas Bawah II (kg),Beras Kualitas Medium I (kg),Beras Kualitas Medium II (kg),Beras Kualitas Super I (kg),Beras Kualitas Super II (kg)
count,429.000000,429.000000,429.000000,429.000000,429.000000,429.000000,429.000000
mean,12309.906760,10957.575758,10881.002331,12515.734266,12431.934732,13812.470862,13203.496503
std,146.735475,269.812495,206.679743,198.218077,157.762496,141.014191,551.706688
min,12050.000000,10500.000000,10650.000000,11400.000000,12150.000000,13500.000000,12650.000000
25%,12150.000000,10750.000000,10750.000000,12350.000000,12250.000000,13650.000000,13000.000000
50%,12350.000000,10850.000000,10750.000000,12550.000000,12450.000000,13850.000000,13100.000000
75%,12350.000000,11100.000000,11100.000000,12650.000000,12500.000000,13950.000000,13500.000000
max,14050.000000,11600.000000,11400.000000,12850.000000,12950.000000,14050.000000,23650.000000


In [ ]:
# Visualisasi Harga Beras kualitas 1 per minggu

datewise["WeekOfYear"]=datewise.index.weekofyear

week_num=[]
weekwise_harga=[]
w=1
for i in list(datewise["WeekOfYear"].unique()):
    weekwise_harga.append(datewise[datewise["WeekOfYear"]==i]["Beras Kualitas Bawah I (kg)"].iloc[-1])
    week_num.append(w)
    w=w+1

fig=go.Figure()
fig.add_trace(go.Scatter(x=week_num, y=weekwise_harga,mode='lines+markers',name='Pertumbuhan Harga Perminggu'))
fig.update_layout(title="Pertumbuhan Harga Beras Kualitas 1 Permingggu",xaxis_title="Minggu",yaxis_title="Harga",legend=dict(x=0,y=1,traceorder="normal"))
fig.show()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series.  To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
# Membagi data train dan Valid

train_ml=datewise.iloc[:int(datewise.shape[0]*0.95)]
valid_ml=datewise.iloc[int(datewise.shape[0]*0.95):]
print(train_ml.shape)
print(valid_ml.shape)

(407, 3)
(22, 3)


In [ ]:
# Membuat Model SVM

svm=SVR(C=1,degree=6,kernel='poly',epsilon=0.01)

#nama kernel di svm : linear, poly, rbf, sigmoid,

In [ ]:
# Fit Model

svm.fit(np.array(train_ml["Days Since"]).reshape(-1,1),np.array(train_ml["Beras Kualitas Bawah I (kg)"]).reshape(-1,1))

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



SVR(C=1, degree=6, epsilon=0.01, kernel='poly')

In [ ]:
# Validasi Prediksi dari awal tanggal

a = prediction_valid_svm=svm.predict(np.array(valid_ml["Days Since"]).reshape(-1,1))

In [ ]:
print(a)

[11667.59246967 11675.36139767 11683.19173973 11691.08388371
 11699.03821914 11707.05513748 11723.27829794 11731.48533228
 11748.0923032  11968.59641744 11978.68220772 11988.84365992
 11999.08122752 12009.39536582 12030.25518686 12040.80179056
 12305.98052161 12343.9354845  12356.76886802 12382.71135542
 12395.82150908 12409.02497299]


In [ ]:
model_scores=[]

In [ ]:
from sklearn.metrics import mean_absolute_error as mae
model_scores.append(np.sqrt(mean_squared_error(valid_ml["Beras Kualitas Bawah I (kg)"],prediction_valid_svm)))
print("Root Mean Square Error (RMSE) untuk Support Vectore Machine: ",np.sqrt(mean_squared_error(valid_ml["Beras Kualitas Bawah I (kg)"],prediction_valid_svm)))
print("Mean Square Error untuk (MSE) Support Vectore Machine: ",mean_squared_error(valid_ml["Beras Kualitas Bawah I (kg)"],prediction_valid_svm))
print("Mean Absolute Error untuk (MAE) Support Vectore Machine: ",mae(valid_ml["Beras Kualitas Bawah I (kg)"],prediction_valid_svm))

Root Mean Square Error untuk Support Vectore Machine:  803.1743187518784
Mean Square Error untuk Support Vectore Machine:  645088.9863025441
Mean Absolute Error untuk Support Vectore Machine:  724.3671521950826


In [ ]:
# Visualisasi Prediksi Harga beras Kualitas 1

plt.figure(figsize=(11,6))
prediction_svm=svm.predict(np.array(datewise["Days Since"]).reshape(-1,1))
fig=go.Figure()
fig.add_trace(go.Scatter(x=datewise.index, y=datewise["Beras Kualitas Bawah I (kg)"],mode='lines+markers',name="Data Harga Training",line=dict(color='red')))
fig.add_trace(go.Scatter(x=datewise.index, y=prediction_svm,mode='lines',name="Vector Machine",line=dict(color='black', dash='dot')))
fig.update_layout(title="Harga Beras Kualitas 1 menggunakan SVM regresi",xaxis_title="Date",yaxis_title="Harga",legend=dict(x=0,y=1,traceorder="normal"))
fig.show()

<Figure size 792x432 with 0 Axes>